In [2]:
#!pip install pandas
#!pip install scikit-learn
#!pip install tabulate

import pandas as pd
from sklearn.linear_model import LinearRegression
from tabulate import tabulate  # Para exibir a tabela formatada

# 1. Leitura dos arquivos CSV
dengue_cases = pd.read_csv('CasosDengue_2020-2024.csv', delimiter=';')
climate_data = pd.read_csv('Media_Climatologica_Anual_2020-2024.csv', delimiter=';')

# 2. Seleção das colunas necessárias e junção dos datasets
dengue_cases = dengue_cases[['Ano', 'Total']]
climate_data.columns = ['Ano', 'Precipitacao', 'Temperatura']

# Juntar os dois datasets com base no ano
data = pd.merge(dengue_cases, climate_data, on='Ano')

# 3. Divisão dos dados de 2020 a 2022 para treinamento
train_data = data[data['Ano'] <= 2022]
test_data = data[data['Ano'] > 2022]

# Separar os dados em X (entrada) e y (saída)
X_train = train_data[['Precipitacao', 'Temperatura']]
y_train = train_data['Total']

# 4. Criação do modelo de regressão linear
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

# 5. Prever casos para 2024, 2025 e 2026
# Adicionando dados fictícios para 2025 e 2026
future_data = pd.DataFrame({
    'Ano': [2025, 2026],
    'Precipitacao': [113.0, 112.0],  # Valores estimados
    'Temperatura': [23.5, 23.3]      # Valores estimados
})

# Combinar os dados de teste (2023 e 2024) com os futuros (2025 e 2026)
test_data_extended = pd.concat([test_data[['Ano', 'Precipitacao', 'Temperatura']], future_data])

# Previsão para os anos 2023 a 2026
X_test = test_data_extended[['Precipitacao', 'Temperatura']]
predictions = regression_model.predict(X_test)

# Tabela com os resultados
result_table = pd.DataFrame({
    'Ano': test_data_extended['Ano'],
    'Previsao_Casos_Dengue': predictions
})

# Adicionar os valores reais para os anos de 2020, 2021 e 2022
for ano in [2020, 2021, 2022]:
    valor_real = data[data['Ano'] == ano]['Total'].values[0]
    result_table = pd.concat([pd.DataFrame({'Ano': [ano], 'Previsao_Casos_Dengue': [valor_real]}), result_table])

# Exibir tabela formatada
print(tabulate(result_table, headers='keys', tablefmt='grid'))


+----+-------+-------------------------+
|    |   Ano |   Previsao_Casos_Dengue |
+====+=======+=========================+
|  0 |  2022 |             1.39383e+06 |
+----+-------+-------------------------+
|  0 |  2021 |        531804           |
+----+-------+-------------------------+
|  0 |  2020 |        944502           |
+----+-------+-------------------------+
|  3 |  2023 |             1.77215e+07 |
+----+-------+-------------------------+
|  4 |  2024 |             1.92993e+07 |
+----+-------+-------------------------+
|  0 |  2025 |             1.93184e+07 |
+----+-------+-------------------------+
|  1 |  2026 |             1.94213e+07 |
+----+-------+-------------------------+
